In [ ]:
import csv
import numpy as np
from itertools import permutations

from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from pprint import pprint

from nn import MLP
from engine import Node

In [ ]:
X, y = make_moons(n_samples=200, noise=0.2)
y = 2*y - 1
plt.scatter(X[:,0], X[:, 1], c=y, cmap='jet')
print('X.shape', X.shape)
print('y.sahpe', y.shape)
print(min(y), max(y))

In [ ]:
model = MLP(2, [32,32], 1)
print(f'Num parameters: {len(model.parameters())}')

In [ ]:
def lossfunc(X, y, model, batch_size=16):
    random_idx = np.random.permutation(X.shape[0])[:batch_size]
    Xb, yb = X[random_idx], y[random_idx]
    
    ypred = [model(x) for x in Xb]
    loss = sum((yout - ygt)**2 for yout, ygt in zip(ypred, yb))    
    
    # L2 Regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    
    total_loss = loss + reg_loss
    
    accuracy = [(yi>0) == (yp>0) for yi, yp in zip(yb, ypred)]
    accuracy_percent = round(100*sum(accuracy)/len(accuracy), 2)
    
    return total_loss, accuracy_percent
     

In [ ]:
steps = 60

y = y.astype('float')

# Gradient Descent Algorithm
for i in range(steps):
    # Forward Pass
    loss, accuracy = lossfunc(X, y, model, batch_size=40)
    learning_rate = -0.005 + 0.0001*(i//8)
    
    if i % int(steps*0.1) == 0 :
        print(i, round(loss.data, 8), accuracy ,round(learning_rate, 6))
    
    model.zero_grad()
    
    # Backward pass
    loss.backprop()
    
    # Update Parameters
    for p in model.parameters():
        p.data += learning_rate * p.grad


In [ ]:
h = 0.2
xmin = X[:,0].min() - 1 
xmax = X[:,0].max() + 1

ymin = X[:,1].min() - 1 
ymax = X[:,1].max() + 1

xx, yy = np.meshgrid(np.arange(xmin, xmax, h), np.arange(ymin, ymax, h))
Xmesh = np.c_[xx.ravel(), yy.ravel()]
inputs = [list(map(Node, xrow)) for xrow in Xmesh]
scores = list(map(model, inputs))

Z = np.array([s.data > 0 for s in scores])
Z = Z.reshape(xx.shape)

In [ ]:
fig, ax = plt.subplots()
ax.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.5)
plt.scatter(X[:,0], X[:, 1], c=y, cmap='jet')